In [1]:
from MEC import Circle, Point, welzl
import numpy as np
import math

def calculate_minimum_enclosing_circle(points):
    """
    Calculate the minimum enclosing circle for a set of points using Welzl's algorithm.
    Returns the center and radius of the circle.
    """
    mec = welzl(points)  
    center=[]
    center.append(mec.C.X)
    center.append(mec.C.Y)
    return center, mec.R

def _rotation_matrix(theta):
    """Creates a 2D rotation matrix for a given angle in radians."""
    return np.array([
        [math.cos(theta), -math.sin(theta)],
        [math.sin(theta), math.cos(theta)]
    ])

def from_rotated_box(xc, yc, w, h, theta, frame_size=None):
    """
    Constructs a rotated bounding box from its center, dimensions, and rotation.

    Args:
        xc (float): x-center coordinate
        yc (float): y-center coordinate
        w (float): box width
        h (float): box height
        theta (float): rotation angle in radians
        frame_size (tuple or None): (width, height) of the frame. If provided, points are normalized.

    Returns:
        list of tuples: Normalized (x, y) coordinates of the rotated bounding box corners
    """
    # Define the rotation matrix
    R = _rotation_matrix(theta)
    
    # Define the local coordinates of the bounding box corners
    local_points = np.array([
        [ w / 2,  h / 2],
        [-w / 2,  h / 2],
        [-w / 2, -h / 2],
        [ w / 2, -h / 2]
    ])  # shape (4, 2)
    
    # Apply rotation
    rotated_points = R.dot(local_points.T).T  # shape (4, 2)
    
    # Apply translation to the center
    rotated_translated_points = rotated_points + np.array([xc, yc])  # shape (4,2)
    
    # If frame_size is provided, normalize the points
    if frame_size is not None:
        rotated_translated_points = rotated_translated_points / np.array(frame_size)
    
    # Convert to list of tuples
    normalized_points = rotated_translated_points.tolist()
    
    return normalized_points

In [2]:
from utils import calculate_euclidean_distance, calculate_real_width
from skeletonization import skeletonize_mask,create_filled_binary_mask, skeletonize_mask, find_longest_path
import cv2
import fiftyone as fo




def process_segmentations(segmentation_path):
    """
    Process the segmentations from the TXT file, calculate the minimum enclosing circle for each prawn.
    """
    segmentations = []
    skeletons=[]
    hulls=[]
    skeletons_straight=[]
    skeletons_straight_2=[]
    seg_closeds=[]
    skeletons_2=[]
    box_diagonal=[] 
    boxes=[]
    masks=[]






    
    # Open the segmentation file and process each line
    with open(segmentation_path, 'r') as file:
        for line in file:
            coords = list(map(float, line.strip().split()))
            binary_mask = create_filled_binary_mask(coords, 640, 640)
            
            
            binary_mask_no= create_filled_binary_mask(coords, 640, 640,gaussian_blur=False) 

            binary_dilated = cv2.dilate(binary_mask_no, np.ones((15, 15), np.uint8), iterations=1)     


            #contour dilated
            contures_dil, _ = cv2.findContours(binary_dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)    

            prawn_conture_dil = max(contures_dil, key=cv2.contourArea)

            coords_contour_dil = np.column_stack(prawn_conture_dil).flatten()

            normalized_coords_bin=[(coords_contour_dil[i]/640, coords_contour_dil[i+1]/640) for i in range(0, len(coords_contour_dil), 2)]  # Extract points (x, y)
            # coords_bin = np.column_stack(np.nonzero(binary_dilated)).flatten()

            # normalized_coords_bin=[(coords_bin[i+1]/640, coords_bin[i]/640) for i in range(0, len(coords_bin), 2)]  # Extract points (x, y)



            masks.append(fo.Polyline(
                points=[normalized_coords_bin],
                closed=True,
                filled=False,
            ))



            # #convert binary mask to normalized coordinates
            # binary_mask_smooth = binary_mask.astype(np.uint8)
            # #x,y coordinates of the mask
            # coords_bin = np.column_stack(np.nonzero(binary_mask_smooth)).flatten()

            # normalized_coords_bin=[(coords_bin[i+1]/640, coords_bin[i]/640) for i in range(0, len(coords_bin), 2)]  # Extract points (x, y)




        
            # #thin the mask
            # thinned=skeletonize_mask(binary_mask)

            # # skeleton = skeletonize_mask(binary_mask)
            # skeleton = thinned
            # skeleton_coords = np.column_stack(np.nonzero(skeleton))
            # normalized_coords,max_length = find_longest_path(skeleton_coords,(640,640),(2988,5312))

            # normalized_coords = [(x, y) for y, x in normalized_coords]  # Convert to (y, x) format

            # #only the first and last points of the skeleton
            # normalized_coords_straight = [normalized_coords[0], normalized_coords[-1]]  

            
            thinned_2=skeletonize_mask(binary_mask_no)

            # skeleton = skeletonize_mask(binary_mask)
            skeleton_2 = thinned_2
            skeleton_coords_2 = np.column_stack(np.nonzero(skeleton_2))
            normalized_coords_2,max_length_2 = find_longest_path(skeleton_coords_2,(640,640),(2988,5312))

            normalized_coords_2 = [(x, y) for y, x in normalized_coords_2]  # Convert to (y, x) format

            #only the first and last points of the skeleton
            normalized_coords_straight_2 = [normalized_coords_2[0], normalized_coords_2[-1]]  






            #convex hull diameter
            contures, _ = cv2.findContours(binary_mask_no, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            
            prawn_conture = max(contures, key=cv2.contourArea)  

            # Compute the minimum area rectangle enclosing the shrimp
            rect = cv2.minAreaRect(prawn_conture)

            (xc, yc), (w, h), theta = rect

# # Convert theta from degrees to radians for FiftyOne
#             theta_radians = np.deg2rad(theta)

            original_size = (640, 640)
            # normalized_polyline = create_rotated_polyline_normalized(xc, yc, w, h, theta_radians, original_size)


            box_points = cv2.boxPoints(rect)
            box_points = np.int0(box_points)

            new_size = (5312, 2988)

            # Scaling factors
            scale_x = new_size[0] / original_size[0]  # 5312 / 640
            scale_y = new_size[1] / original_size[1]  # 2988 / 640

            scale_x = new_size[0] / original_size[0]  # 5312 / 640 = 8.3
            scale_y = new_size[1] / original_size[1]  # 2988 / 640 ≈ 4.66875

            # Scale the center coordinates and dimensions
            scaled_xc = xc * scale_x
            scaled_yc = yc * scale_y
            scaled_w = w * scale_x
            scaled_h = h * scale_y

            # Calculate the rotated bounding box points
            theta_radians = np.deg2rad(theta)


            uniform_scale = min(scale_x, scale_y)
            scaled_w = w * uniform_scale
            scaled_h = h * uniform_scale
            scaled_xc = xc * uniform_scale
            scaled_yc = yc * uniform_scale

            # Generate normalized bounding box points
            points = from_rotated_box(
                scaled_xc, scaled_yc, scaled_w, scaled_h, theta_radians, frame_size=new_size
            )

            # points=from_rotated_box (scaled_xc, scaled_yc, scaled_w, scaled_h, theta_radians, frame_size=(5312, 2988))



            box_points_scaled = np.array([(point[0] * scale_x, point[1] * scale_y) for point in box_points])

            width= calculate_euclidean_distance(box_points_scaled[0], box_points_scaled[1])
            height= calculate_euclidean_distance(box_points_scaled[1], box_points_scaled[2])

            max_length_box=max(width,height)

            # print(f"Original Center: ({xc}, {yc})")
            # print(f"Scaled Center: ({scaled_xc}, {scaled_yc})")
            # print(f"Original Dimensions: (w={w}, h={h})")
            # print(f"Scaled Dimensions: (w={scaled_w}, h={scaled_h})")
            # print(f"Rotation Angle (radians): {theta_radians}")
            # print(f"Normalized Bounding Box Points: {points}")
            # print(f"Max Length Box (pixels): {max_length_box}")
                    

            # normalized_bounding_box = [
            #         (x / new_size[0], y / new_size[1]) for x, y in box_points_scaled]            
            # Extract points (x, y) 
            box=fo.Polyline(
                points=[points],
                closed=True,
                filled=False,
                max_length=max_length_box
            )
            # box=fo.Polyline.from_rotated_box(

            #     xc=xc/640,
            #     yc=yc/640,
            
            #     w=w*scale_x,
            #     h=h*scale_y,
            #     theta =theta_radians,
            

            # )

            boxes.append(box)


            hull_points = cv2.convexHull(prawn_conture, returnPoints=True)


            
# Scaling factors to convert from 640x640 to 5312x2988
            scale_x = 5312 / 640
            scale_y = 2988 / 640

        # Scale the points to the new resolution
            scaled_hull_points = []
            for point in hull_points:
                x, y = point[0]
                scaled_x = x * scale_x
                scaled_y = y * scale_y
                scaled_hull_points.append([scaled_x, scaled_y])

            # Convert to numpy array for easier handling
            scaled_hull_points = np.array(scaled_hull_points, dtype=np.float32)

            # Now, find the maximum Euclidean distance (convex hull diameter) using the scaled points
            max_distance = 0
            point1 = None
            point2 = None

            # Loop through all pairs of scaled points to find the maximum distance
            for i in range(len(scaled_hull_points)):
                for j in range(i + 1, len(scaled_hull_points)):
                    distance = calculate_euclidean_distance(scaled_hull_points[i], scaled_hull_points[j])
                    if distance > max_distance:
                        max_distance = distance
                        point1 = scaled_hull_points[i]
                        point2 = scaled_hull_points[j]

            # The result is max_distance (in pixels) in the 5312x2988 image


            normalzied_points_hull = [(point1[0]/5312, point1[1]/2988), (point2[0]/5312, point2[1]/2988)]  # Extract points (x, y)

            hull=fo.Polyline(
                points=[normalzied_points_hull],
                closed=False,
                filled=False,
                max_length=max_distance
            )

            # skeleton_straight=fo.Polyline(
            #     points=[normalized_coords_straight],
            #     closed=False,
            #     filled=False,
            #     max_length=max_length
            # )
            # skeletons_straight.append(skeleton_straight)

            skeleton_straight_2=fo.Polyline(
                points=[normalized_coords_straight_2],
                closed=False,
                filled=False,
                max_length=max_length_2,
                
            )
            skeletons_straight_2.append(skeleton_straight_2)




            hulls.append(hull)

            # skeleton=fo.Polyline(
            #     points=[normalized_coords],
            #     closed=False,
            #     filled=False,
            #     max_length=max_length
            # )

            # skeletons.append(skeleton)
            
            skeleton_2=fo.Polyline( 
                points=[normalized_coords_2],
                closed=False,
                filled=False,
                max_length=max_length_2)
            skeletons_2.append(skeleton_2)
              # Convert the line to a list of floats
            normalzied_points = [(coords[i]/640, coords[i + 1]/640) for i in range(0, len(coords), 2)]  # Extract points (x, y)
            points = [Point(x*5312, y*2988) for x, y in normalzied_points] 
            

            
             # Convert to Point objects    
            # Calculate the minimum enclosing circle (center and radius)
            center, radius = calculate_minimum_enclosing_circle(points)
            diameter = radius * 2

            segmentation = fo.Polyline(
                points=[normalzied_points],
                closed=True,
                filled=False,
                diameter=diameter,
                center=center,
                max_length_skeleton=max_length_2,
                max_length_hull=max_distance,
                max_length_box=max_length_box
            )

            #smooth segmentation  wirh closing
            # seg_closed=fo.Polyline(
            #     points=[normalized_coords_bin],
            #     closed=True,
            #     filled=False,
            #     max_length=max_length
            # )

            # seg_closeds.append(seg_closed)                


            segmentations.append(segmentation)







                     # Store the segmentation information (center, radius, and diameter)

    return segmentations,skeletons,hulls, skeletons_straight,seg_closeds,skeletons_2,skeletons_straight_2,boxes,masks

In [3]:
from utils import calculate_euclidean_distance, calculate_real_width
from skeletonization import skeletonize_mask,create_filled_binary_mask, skeletonize_mask, find_longest_path
import cv2
import fiftyone as fo
import numpy as np

def process_segmentations(segmentation_path):
    """
    Process the segmentations from the TXT file, calculate the minimum enclosing circle for each prawn.
    """
    segmentations = []
    skeletons=[]
    hulls=[]
    skeletons_straight=[]
    skeletons_straight_2=[]
    seg_closeds=[]
    skeletons_2=[]
    box_diagonal=[] 
    boxes=[]
    masks=[]






    
    # Open the segmentation file and process each line
    with open(segmentation_path, 'r') as file:
        for line in file:
            coords = list(map(float, line.strip().split()))
            binary_mask = create_filled_binary_mask(coords, 640, 640)
            
            
            binary_mask_no= create_filled_binary_mask(coords, 640, 640,gaussian_blur=False) 

            binary_dilated = cv2.dilate(binary_mask_no, np.ones((15, 15), np.uint8), iterations=1)     


            #contour dilated
            contures_dil, _ = cv2.findContours(binary_dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)    

            prawn_conture_dil = max(contures_dil, key=cv2.contourArea)

            coords_contour_dil = np.column_stack(prawn_conture_dil).flatten()

            normalized_coords_bin=[(coords_contour_dil[i]/640, coords_contour_dil[i+1]/640) for i in range(0, len(coords_contour_dil), 2)]  # Extract points (x, y)
            # coords_bin = np.column_stack(np.nonzero(binary_dilated)).flatten()

            # normalized_coords_bin=[(coords_bin[i+1]/640, coords_bin[i]/640) for i in range(0, len(coords_bin), 2)]  # Extract points (x, y)



            masks.append(fo.Polyline(
                points=[normalized_coords_bin],
                closed=True,
                filled=False,
            ))



            # #convert binary mask to normalized coordinates
            # binary_mask_smooth = binary_mask.astype(np.uint8)
            # #x,y coordinates of the mask
            # coords_bin = np.column_stack(np.nonzero(binary_mask_smooth)).flatten()

            # normalized_coords_bin=[(coords_bin[i+1]/640, coords_bin[i]/640) for i in range(0, len(coords_bin), 2)]  # Extract points (x, y)




        
            # #thin the mask
            # thinned=skeletonize_mask(binary_mask)

            # # skeleton = skeletonize_mask(binary_mask)
            # skeleton = thinned
            # skeleton_coords = np.column_stack(np.nonzero(skeleton))
            # normalized_coords,max_length = find_longest_path(skeleton_coords,(640,640),(2988,5312))

            # normalized_coords = [(x, y) for y, x in normalized_coords]  # Convert to (y, x) format

            # #only the first and last points of the skeleton
            # normalized_coords_straight = [normalized_coords[0], normalized_coords[-1]]  

            
            thinned_2=skeletonize_mask(binary_mask_no)

            # skeleton = skeletonize_mask(binary_mask)
            skeleton_2 = thinned_2
            skeleton_coords_2 = np.column_stack(np.nonzero(skeleton_2))
            normalized_coords_2,max_length_2 = find_longest_path(skeleton_coords_2,(640,640),(2988,5312))

            normalized_coords_2 = [(x, y) for y, x in normalized_coords_2]  # Convert to (y, x) format

            #only the first and last points of the skeleton
            normalized_coords_straight_2 = [normalized_coords_2[0], normalized_coords_2[-1]]  






            #convex hull diameter
            contures, _ = cv2.findContours(binary_mask_no, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            
            prawn_conture = max(contures, key=cv2.contourArea)  

            # Compute the minimum area rectangle enclosing the shrimp
            rect = cv2.minAreaRect(prawn_conture)
            box_points = cv2.boxPoints(rect)
            box_points = np.int0(box_points)

            original_size = (640, 640)
            new_size = (5312, 2988)

            # Scaling factors
            scale_x = new_size[0] / original_size[0]  # 5312 / 640
            scale_y = new_size[1] / original_size[1]  # 2988 / 640

            box_points_scaled = np.array([(point[0] * scale_x, point[1] * scale_y) for point in box_points])

            width= calculate_euclidean_distance(box_points_scaled[0], box_points_scaled[1])
            height= calculate_euclidean_distance(box_points_scaled[1], box_points_scaled[2])

            max_length_box=max(width,height)


           
            # Convert theta from degrees to radians for FiftyOne
            theta_radians = np.deg2rad(rect[2])
            # normalized_bounding_box = [(box_points[i][0]/640, box_points[i][1]/640) for i in range(0, len(box_points))] 
            
            image_center_x = 640 / 2

            xc_adjusted = rect[0][0] - image_center_x

            # Extract points (x, y) 
            box=fo.Polyline.from_rotated_box(
                xc=rect[0][0] ,
                yc=rect[0][1],
                w=rect[1][0],
                h=rect[1][1],
                theta =theta_radians,
                frame_size=(640, 640)

            )


            boxes.append(box)


            hull_points = cv2.convexHull(prawn_conture, returnPoints=True)


            
# Scaling factors to convert from 640x640 to 5312x2988
            scale_x = 5312 / 640
            scale_y = 2988 / 640

        # Scale the points to the new resolution
            scaled_hull_points = []
            for point in hull_points:
                x, y = point[0]
                scaled_x = x * scale_x
                scaled_y = y * scale_y
                scaled_hull_points.append([scaled_x, scaled_y])

            # Convert to numpy array for easier handling
            scaled_hull_points = np.array(scaled_hull_points, dtype=np.float32)

            # Now, find the maximum Euclidean distance (convex hull diameter) using the scaled points
            max_distance = 0
            point1 = None
            point2 = None

            # Loop through all pairs of scaled points to find the maximum distance
            for i in range(len(scaled_hull_points)):
                for j in range(i + 1, len(scaled_hull_points)):
                    distance = calculate_euclidean_distance(scaled_hull_points[i], scaled_hull_points[j])
                    if distance > max_distance:
                        max_distance = distance
                        point1 = scaled_hull_points[i]
                        point2 = scaled_hull_points[j]

            # The result is max_distance (in pixels) in the 5312x2988 image


            normalzied_points_hull = [(point1[0]/5312, point1[1]/2988), (point2[0]/5312, point2[1]/2988)]  # Extract points (x, y)

            hull=fo.Polyline(
                points=[normalzied_points_hull],
                closed=False,
                filled=False,
                max_length=max_distance
            )

            # skeleton_straight=fo.Polyline(
            #     points=[normalized_coords_straight],
            #     closed=False,
            #     filled=False,
            #     max_length=max_length
            # )
            # skeletons_straight.append(skeleton_straight)

            skeleton_straight_2=fo.Polyline(
                points=[normalized_coords_straight_2],
                closed=False,
                filled=False,
                max_length=max_length_2,
                
            )
            skeletons_straight_2.append(skeleton_straight_2)




            hulls.append(hull)

            # skeleton=fo.Polyline(
            #     points=[normalized_coords],
            #     closed=False,
            #     filled=False,
            #     max_length=max_length
            # )

            # skeletons.append(skeleton)
            
            skeleton_2=fo.Polyline( 
                points=[normalized_coords_2],
                closed=False,
                filled=False,
                max_length=max_length_2)
            skeletons_2.append(skeleton_2)
              # Convert the line to a list of floats
            normalzied_points = [(coords[i]/640, coords[i + 1]/640) for i in range(0, len(coords), 2)]  # Extract points (x, y)
            points = [Point(x*5312, y*2988) for x, y in normalzied_points] 
            

            
             # Convert to Point objects    
            # Calculate the minimum enclosing circle (center and radius)
            center, radius = calculate_minimum_enclosing_circle(points)
            diameter = radius * 2

            segmentation = fo.Polyline(
                points=[normalzied_points],
                closed=True,
                filled=False,
                diameter=diameter,
                center=center,
                max_length_skeleton=max_length_2,
                max_length_hull=max_distance,
                max_length_box=max_length_box
            )

            #smooth segmentation  wirh closing
            # seg_closed=fo.Polyline(
            #     points=[normalized_coords_bin],
            #     closed=True,
            #     filled=False,
            #     max_length=max_length
            # )

            # seg_closeds.append(seg_closed)                


            segmentations.append(segmentation)







                     # Store the segmentation information (center, radius, and diameter)

    return segmentations,skeletons,hulls, skeletons_straight,seg_closeds,skeletons_2,skeletons_straight_2,boxes,masks

In [4]:
import fiftyone.core.labels as fol
from tqdm import tqdm
import fiftyone as fo
import os

def process_images(image_paths, prediction_folder_path, dataset):
    print("Processing images...")
    
    """
    Processes images by matching segmentation with bounding boxes and calculating prawn sizes.
    """
    for image_path in tqdm(image_paths):
        # filename = os.path.splitext(os.path.basename(image_path))[0]
        

           
        # prediction_txt_path = os.path.join(prediction_folder_path, f"{os.path.basename(image_path).split('.')[0]}_segmentations.txt")

        core_name = os.path.splitext(os.path.basename(image_path))[0]

        # Construct the path to the corresponding segmentation file
        prediction_txt_path = os.path.join(prediction_folder_path, f"{core_name}_segmentations.txt")

        # core_name=filename.split('.')[0]
        # # Construct the path to the prediction (segmentation) file
        # prediction_txt_path = os.path.join(prediction_folder_path, f"{core_name}_segmentations.txt")
        # if not os.path.exists(prediction_txt_path):
        #     print(f"No segmentation file found for {filename}")
        #     continue


        # Parse the segmentations to get the minimum enclosing circles
        segmentations,skeletons,hulls,skeletons_straight,seg_closeds,skeletons_2,skeletons_straight_2,boxes,masks = process_segmentations(prediction_txt_path)

        # Save the modified image (with circles drawn)

        # Create a new sample for FiftyOne
        sample = fo.Sample(filepath=image_path)

        # Iterate over each bounding box in the filtered data
        sample["segmentations"] = fol.Polylines(polylines=segmentations)

        # sample["skeletons"] = fol.Polylines(polylines=skeletons)

        sample["hulls"] = fol.Polylines(polylines=hulls)    

        # sample["skeletons_straight"] = fol.Polylines(polylines=skeletons_straight)

        # sample['seg_closeds']=fol.Polylines(polylines=seg_closeds)
        
        sample['skeletons_no_smooth']=fol.Polylines(polylines=skeletons_2)

        sample["skeletons_straight_no_smooth"] = fol.Polylines(polylines=skeletons_straight_2)

        sample['boxes']=fol.Polylines(polylines=boxes)

        sample['masks']=fol.Polylines(polylines=masks)
        # Add the processed sample to the FiftyOne dataset
        dataset.add_sample(sample)

In [5]:
import math
def process_detection_by_circle(segmentation):
    """
    Process the prawn detection based on the enclosing circle's diameter.
    Update the filtered dataframe with the real-world size of the prawn.
    """
    
    # Fetch height in mm and other metadata
    height_mm =500-30
    #focal length based on pond type
    
    focal_length = 24.72


    # focal_length = 24.22  # Camera focal length
    pixel_size = 0.00716844  # Pixel size in mm

    poly=segmentation

    fov=75
    FOV_width=2*height_mm*math.tan(math.radians(fov/2))


    # Get the diameter of the circle in pixels
    predicted_diameter_pixels = poly['diameter']


    predicted_skeleton_length=poly['max_length_skeleton']  

    predicted_hull_length=poly['max_length_hull']

     
    predicted_box_length=poly['max_length_box']

    # Calculate the real-world prawn size using the box
    real_length_mm_box = calculate_real_width(focal_length, height_mm, predicted_box_length, pixel_size) 


    hull_length_cm = calculate_real_width(focal_length, height_mm, predicted_hull_length, pixel_size)    

    # Calculate the real-world prawn size using the enclosing circle's diameter
    real_length_cm = calculate_real_width(focal_length, height_mm, predicted_diameter_pixels, pixel_size)

    ske_length_cm = calculate_real_width(focal_length, height_mm, predicted_skeleton_length, pixel_size)    


    hull_length_fov=FOV_width*predicted_hull_length/5312
    diameter_length_fov=FOV_width*predicted_diameter_pixels/5312
    skeleton_length_fov=FOV_width*predicted_skeleton_length/5312

    box_length_fov=FOV_width*predicted_box_length/5312

    true_length=143
    error_percentage_MEC_fov = abs(diameter_length_fov - true_length) / true_length * 100

    error_percentage_hull_fov = abs(hull_length_fov - true_length) / true_length * 100

    error_percentage_skeleton_fov = abs(skeleton_length_fov - true_length) / true_length * 100  

    error_percentage = abs(real_length_cm - true_length) / true_length * 100

    error_percentage_skeleton = abs(ske_length_cm - true_length) / true_length * 100    

    error_percentage_hull = abs(hull_length_cm - true_length) / true_length * 100

    error_percentage_box_fov = abs(box_length_fov - true_length) / true_length * 100

    closest_detection_label = f'true length: {true_length:.2f}mm, MPError_hull: {error_percentage_hull_fov:.2f}%, , pred length: {hull_length_fov:.2f}mm ,error percentage skeleton: {error_percentage_skeleton_fov:.2f}%, , pred length: {skeleton_length_fov:.2f}cm, error percentage box: {error_percentage_box_fov:.2f}%, pred length: {box_length_fov:.2f}mm, '
    # Update the filtered dataframe with the real-world size of the prawn

    poly.label = closest_detection_label 
    return poly
  

In [6]:
molt_image_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\molt\molt-19-9\unditorted_resized"
molt_prediction=r'C:\Users\gbo10\OneDrive\measurement_paper_images\molt\molt-19-9\unditorted_resized'

import fiftyone as fo
dataset = fo.Dataset("molt", overwrite=True)

# Load the dataset
image_paths = [os.path.join(molt_image_path, image) for image in os.listdir(molt_image_path) if image.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]
prediction_paths_text = [os.path.join(molt_prediction, txt) for txt in os.listdir(molt_prediction) if txt.endswith('.txt')]

# Process images

process_images(image_paths, molt_prediction, dataset)

# Process segmentations
for sample in dataset:
    # Access the polylines for each sample
    for i, segmentation in enumerate(sample["segmentations"].polylines):
        # Process and modify the segmentation
        updated_segmentation = process_detection_by_circle(segmentation)

        # Save the updated segmentation back into the sample
        sample["segmentations"].polylines[i] = updated_segmentation

        sample.save()
# Launch FiftyOne session
session = fo.launch_app(dataset)

Processing images...


  0%|          | 0/6 [00:00<?, ?it/s]C:\Users\gbo10\AppData\Local\Temp\ipykernel_39168\370280592.py:113: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box_points = np.int0(box_points)
100%|██████████| 6/6 [00:01<00:00,  3.85it/s]



Could not connect session, trying again in 10 seconds



RuntimeError: Client is not connected

In [27]:
import cv2
import numpy as np

# Load the image
image = cv2.imread(r"C:\Users\gbo10\OneDrive\research\06.05.2024\vlcsnap-2024-09-20-23h55m42s785.png")

# Step 1: Convert to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


white_mask=cv2.inRange(image, (100, 100, 100), (255, 255, 255))

cv2.imwrite("white_mask.png", white_mask)    



very_white_mask = cv2.inRange(image, (150, 150, 150), (255, 255, 255))


image[very_white_mask == 255] = [0, 0, 0]  # Change white to gray


cv2.imwrite("very_white_mask.png", image)


gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


_, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

# Save the result
cv2.imwrite("binary_image.png", binary)


cv2.imwrite("gray.png", gray)


white_mask_from_binary = cv2.bitwise_not(binary)  # Invert the binary mask to get white areas

cv2.imwrite("white_mask_from_binary.png", white_mask_from_binary)


gray_color = np.array([79, 105, 122])  # RGB value for gray
# image[white_mask == 255] = gray_color

# Step 2: Threshold the image to isolate black areas (under prawns)
# Adjust threshold values depending on how black the segments are
_, black_mask = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)  # Isolating black areas (threshold of 50)


cv2.imwrite("black_mask.png", black_mask)

# Step 3: Create a bluish image to apply to black areas
bluish_image = image.copy()
bluish_image[:] = [212, 156, 31]  # BGR for blue


#white mask on the black segments
# white_on_black = cv2.bitwise_and(white_mask_from_binary, black_mask)  # Isolate white areas on the black segments

# cv2.imwrite("white_on_black.png", white_on_black)


# # Step 6: Convert white to gray in those areas
# # gray_color = np.array([128, 128, 128])  # RGB value for gray
# image[white_on_black==0] = gray_color  # Change white on black to gray


image[white_mask == 255] = gray_color

# Step 4: Apply bluish color to the black regions
image[black_mask == 255] = bluish_image[black_mask == 255]





# Step 5: Isolate white areas (likely prawns) within the black segments
# Detect white pixels in the original image (on the black segments only)
# white_on_black = cv2.bitwise_and(white_mask, black_mask)  # Isolate white areas on the black segments

# Step 6: Convert white to gray in those areas
# gray_color = np.array([128, 128, 128])  # RGB value for gray
# image[white_on_black == 255] = gray_color  # Change white on black to gray

# Step 7: Display the final image
# cv2.imshow("Result", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Save the result
cv2.imwrite("processed_image.png", image)


True